In [1]:
# reference data gathering 

import csv
import json
import pandas as pd
import re
import logging
import os
import random
import sys
from heapq import nlargest
import openai
import torch
from tenacity import retry, stop_after_attempt, wait_exponential
import pandas as pd


with open('../data_set_v1.0/author_json.json', 'r',encoding='utf-8') as f:
    authors = json.load(f)
print(len(authors))

with open('../data_set_v1.0/paper_json.json', 'r',encoding='utf-8') as f:
    papers = json.load(f)
print(len(papers))
# sampled papers to 500 
count = 0
max_paper = 100
author_paper_count = []
paper_author_count = []
papers_sampled = {}
authors_sampled = {}
for i in papers:
    if count < 100:
        flag = True
        for author in papers[i]["authors"]:
            if len(authors[author]["papers"])<5:
                flag = False
        if flag and len(papers[i]["authors"]) > 1:
            paper_author_count.append(len(papers[i]["authors"]))
            papers_sampled[i] = papers[i]
            papers_sampled[i]["author_history"] = {}
            
            for author in papers[i]["authors"]:
                papers_sampled[i]["author_history"][author] = {}
                papers_sampled[i]["author_history"][author]["authors_history_text"] = [papers[i]["abstract"]]
                papers_sampled[i]["author_history"][author]["authors_history_dict"] = [{"Abstract":"","Title":""}]
                paper_count_tmp = 0
                for paper_tmp in authors[author]["papers"]:
                    if paper_count_tmp < max_paper and paper_tmp["title"] != papers[i]["title"]:
                        papers_sampled[i]["author_history"][author]["authors_history_text"].append("Abstract: "+paper_tmp["abstract"]+"\nTitle: "+paper_tmp["title"])
                        papers_sampled[i]["author_history"][author]["authors_history_dict"].append({"Abstract":paper_tmp["abstract"],"Title":paper_tmp["title"]})
                        paper_count_tmp = paper_count_tmp + 1
                author_paper_count.append(len(authors[author]["papers"]))
                authors_sampled[author] = authors[author]
            count = count + 1
print("papers count after sampled: ",len(papers_sampled))
print("authors count after sampled: ",len(authors_sampled))


7931
3275
papers count after sampled:  100
authors count after sampled:  243


In [2]:
#!/usr/bin/env python3


import argparse
import csv
import os
from requests import Session
from typing import Generator, TypeVar
import requests

S2_API_KEY = "mEwrkvDmRc9KFXu76cAAk16KJxHNNUuf9tYaot1g"


def get_paper_batch( ids, fields):
    params = {
        'fields': fields
    }
    headers = {
        'X-API-KEY': S2_API_KEY,
    }
    body = {
        'ids': ids,
    }

    # https://api.semanticscholar.org/api-docs/graph#tag/Paper-Data/operation/post_graph_get_papers
    with requests.post('https://api.semanticscholar.org/graph/v1/paper/batch',
                       params=params,
                       headers=headers,
                       json=body
                       ) as response:
        return response.content
paper2citations = {}
paperIds = [i for i in papers_sampled]
paper2citations = get_paper_batch(paperIds,["references"])
paper2citations = json.loads(paper2citations)

id2citations = {}
for paper in paper2citations:
    id2citations[paper["paperId"]] = paper["references"]

In [3]:
input_history = pd.read_csv('../naive_method_v1.0/paper_to_author_cot_5_examples_.tsv', sep='\t', header=None, names=["paper_index","author_index","example_index","result","abstract","title"])
user_keywords_topics = pd.read_csv('../naive_method_v1.0/author_profiling_20_examples_.tsv', sep='\t', header=None, names=["author_index","keywords","topics"])
paper_rq = pd.read_csv('../naive_method_v1.0/paper_research_questions.tsv', sep='\t', header=None, names=["paper_index","research_questions"])
user_interests = pd.read_csv('../task1/author_interests_gold.tsv', sep='\t', header=None, names=["author_index","interests"])

id2authorTopics = {}
id2RQ = {}
id2authorInterests = {}
id2ICLexamples = {}
for i in range(len(user_keywords_topics["author_index"])):
    id2authorTopics[user_keywords_topics["author_index"][i]] = {"keywords":user_keywords_topics["keywords"][i],"topics":user_keywords_topics["topics"][i]}

for i in range(len(paper_rq["paper_index"])):
    id2RQ[paper_rq["paper_index"][i]] = paper_rq["research_questions"][i]

for i in range(len(user_interests["author_index"])):
    id2authorInterests[user_interests["author_index"][i]] = {"interests":user_interests["interests"][i]}

for i in range(len(input_history["paper_index"])):
    paperId = input_history["paper_index"][i]
    authorId = input_history["author_index"][i]
    if paperId not in id2ICLexamples:
        id2ICLexamples[paperId] = {}
    if authorId not in id2ICLexamples[paperId]:
        id2ICLexamples[paperId][authorId]=[]
    id2ICLexamples[paperId][authorId].append({"abstract":input_history["abstract"][i],"title":input_history["title"][i],"result":input_history["result"][i]})



In [14]:
# Task-2 Zero-shot
error_list=[]
i = 0
openai.api_key = "31b8638c9eea48709a596501490f9e88"
openai.api_base = "https://gersteinbiocodeeval-eastus2.openai.azure.com/"
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future
key_bundles = [
    ('aaccba8e27374383beb397ecdc615ee5', "https://biocodeeval-openai.openai.azure.com/"),
    ('31b8638c9eea48709a596501490f9e88', "https://gersteinbiocodeeval-eastus2.openai.azure.com/"),
    ('aaccba8e27374383beb397ecdc615ee5', "https://biocodeeval-openai.openai.azure.com/")
]

    # prompt_construction v1.6
def construct_prompt_zero_shot_1(reference_dict,paperId):
    ref_example = []
    for index,item in enumerate(reference_dict):
        if index <50:
            ref_example.append("Reference Paper #"+str(index)+"\n"+"Title: "+item["title"])
            
    prompt_str = ""

    prompt_str = prompt_str + "You're a researcher working on a new paper.\n"
    prompt_str = prompt_str + "Now you have come up with the idea for this work:\n"
    prompt_str = prompt_str + "Idea: "+ papers_sampled[paperId]["title"] + "\n"
    
    prompt_str = prompt_str + "With the focus on the following research questions:\n"
    prompt_str = prompt_str + "Research questions: "+ id2RQ[paperId]+ "\n"
    prompt_str = prompt_str + "Fully consider both the the idea and the research questions to write a abstract for this paper:\n"
    prompt_str = prompt_str + "Abstract: "

    return prompt_str

def construct_prompt_few_shot(reference_dict,paperId,authorDicts):
    prompt_str = ""
    keywords = ""
    interests = ""
    authorId = ""
    topics = ""
    authorDict = []
    ref_example = []
    history_examples = []
    for i in authorDicts:
        authorId = i
        authorDict = authorDicts[authorId]
        break

    for index,item in enumerate(reference_dict):
        if index <50:
            ref_example.append("Reference Paper #"+str(index)+"\n"+"Title: "+item["title"])
    if authorId in id2authorInterests:
        if "interests" in id2authorInterests[authorId]:
            interests =  id2authorInterests[authorId]["interests"]
    if "keywords" in id2authorTopics[authorId]:
        keywords = str(id2authorTopics[authorId]["keywords"])
    if "topics" in id2authorTopics[authorId]:
        topics = str(id2authorTopics[authorId]["topics"])
    for index,item in enumerate(authorDict):
        if index <5:
            # history_examples.append("Example #"+str(index)+"\nAbstract: "+item["abstract"]+"\nStyle: "+item["result"]+"\nTitle: "+item["title"])
            history_examples.append("History Paper #"+str(index)+"\nTitle: "+item["title"]+"\nAbstract: "+item["abstract"])
            # history_examples.append("History Paper #"+str(index)+": "+item["title"])

    
    prompt_str = prompt_str + "Assuming you are an expert researcher in these areas:\n"
    prompt_str = prompt_str + " ".join(interests)
    prompt_str = prompt_str + "Here're some titles and paper abstracts that you've written.\n"
    prompt_str = prompt_str + "\n".join(history_examples) + "\n"


    prompt_str = prompt_str + "Now you're working on a new paper.\n"
    # prompt_str = prompt_str + "You have gathered some related science papers for reference, each of them contains a title.\n"
    # prompt_str = prompt_str + "\n".join(ref_example) + "\n"
    
    prompt_str = prompt_str + "You have confirmed to work on the following topic in this paper:\n"
    prompt_str = prompt_str + "Idea: "+ papers_sampled[paperId]["title"] + "\n"

    prompt_str = prompt_str + "With the focus on the following research questions:\n"
    prompt_str = prompt_str + "Research questions: "+ id2RQ[paperId]+ "\n"

    prompt_str = prompt_str + "Please fully consider both the information above and imitate your writing styles of the previous abstracts, to write a abstract for this paper, keeping it faithful and short:\n"
    prompt_str = prompt_str + "Abstract: "

    return prompt_str


def construct_prompt_few_shot_for_multi_agent_input(reference_dict,paperId,authorId,authorDict):
    prompt_str = ""
    keywords = ""
    interests = ""
    topics = ""
    ref_example = []
    history_examples = []

    for index,item in enumerate(reference_dict):
        if index <50:
            ref_example.append("Reference Paper #"+str(index)+"\n"+"Title: "+item["title"])
    if authorId in id2authorInterests:
        if "interests" in id2authorInterests[authorId]:
            interests =  id2authorInterests[authorId]["interests"]
    for index,item in enumerate(authorDict):
        if index <5:
            # history_examples.append("Example #"+str(index)+"\nAbstract: "+item["abstract"]+"\nStyle: "+item["result"]+"\nTitle: "+item["title"])
            history_examples.append("History Paper #"+str(index)+": "+item["title"])
            # history_examples.append("Example #"+str(index)+"\nAbstract: "+item["abstract"]+"\nTitle: "+item["title"])

    
    prompt_str = prompt_str + "Assuming you are an expert researcher in these areas:\n"
    prompt_str = prompt_str + " ".join(interests)
    prompt_str = prompt_str + "Here're some titles and paper abstracts that you've written.\n"
    prompt_str = prompt_str + "\n".join(history_examples) + "\n"


    prompt_str = prompt_str + "Now you're working on a new paper about this idea.\n"
    prompt_str = prompt_str + "Idea: "+ papers_sampled[paperId]["title"] + "\n"

    prompt_str = prompt_str + "And you are focusing to solve the following research questions:\n"
    prompt_str = prompt_str + "Research questions: "+ id2RQ[paperId]+ "\n"

    prompt_str = prompt_str + "Please fully consider both the information above and imitate your writing styles of the previous abstracts, to write a abstract for this paper, keeping it faithful and short:\n"
    prompt_str = prompt_str + "Abstract: "
    return prompt_str

def run_openai(prompt,key_bundles,i,indice):
    my_dict = {}
    my_dict["role"] = "user"
    my_dict["content"] = prompt
    l = []
    l.append(my_dict)
    result = ""
    try:
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
        result = response.choices[0].message["content"].replace('\n', ' ')
        # print(indice+"\t"+result+"\n")
        return result
    except openai.error.Timeout:
        i = i+1
        print("Timeout",indice)
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        try:
            response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
            result = response.choices[0].message["content"].replace('\n', ' ')
            # print(indice+"\t"+result+"\n")
            return result
        except openai.error.Timeout:
            i = i+1
            print("Timeout",indice)
            key_bundle = key_bundles[i%3]
            openai.api_key, openai.api_base = key_bundle
            try:
                response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
                result = response.choices[0].message["content"].replace('\n', ' ')
                # print(indice+"\t"+result+"\n")
                return result
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                error_list.append(i)
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            error_list.append(i)
        #print(result)
    except openai.error.InvalidRequestError:
        print("InvalidRequestError",indice)
        error_list.append(i)
        #print(result)
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        error_list.append(i)
    return result

id2result = {}
# with open("few_shot_results_single_author.tsv", "wb", buffering=0) as out_file:
#     for indice,paperId in enumerate(id2citations):
#         if paperId in id2citations and paperId in papers_sampled:       
#             # prompt = construct_prompt_zero_shot_1(id2citations[paperId],paperId)
#             prompt = construct_prompt_few_shot(id2citations[paperId],paperId,id2ICLexamples[paperId])
#             # print(prompt)
#             # break
#             result = run_openai(prompt,key_bundles,i,indice)
#             print(indice, result)
#             id2result[paperId] = result
#             write_str = bytes(str(paperId)+"\t"+result+"\n", 'utf-8')
#             out_file.write(write_str)

# id2result_multi_agent = id2result

with open("few_shot_1_title_single_author_for_multi_agents_input.tsv", "wb", buffering=0) as out_file:

    for indice,paperId in enumerate(id2ICLexamples):
        # prompt = construct_prompt_zero_shot_1(id2citations[paperId],paperId)
        if paperId in id2citations:
            id2result[paperId] = []
            for authorId in id2ICLexamples[paperId]:
                prompt = construct_prompt_few_shot_for_multi_agent_input(id2citations[paperId],paperId,authorId,id2ICLexamples[paperId][authorId])
                # print(prompt)
                # break
                result = run_openai(prompt,key_bundles,i,indice)
                print(indice, result)
                id2result[paperId].append({authorId:result})
                write_str = bytes(str(paperId)+"\t"+str(authorId)+"\t"+result+"\n", 'utf-8')
                out_file.write(write_str)

0 Model-Driven Software Engineering (MDSE) has the potential to improve the daily practice of software professionals by providing a more efficient and effective approach to software development. This paper explores the basic principles and techniques of MDSE that can be quickly understood and applied for immediate benefit. We also examine the technical aspects of MDSE, including the building of a domain-specific modeling language and the description of Model-to-Text and Model-to-Model transformations. Additionally, we discuss how the integration of MDSE in existing development processes can be effectively achieved. By providing practical guidance on the implementation of MDSE, this paper aims to help software professionals improve their software development practices.
0 Model-Driven Software Engineering (MDSE) or Model-Driven Engineering (MDE) has the potential to improve the daily practice of software professionals. This paper aims to answer three research questions: 1) How can model-

In [6]:
author_roles = pd.read_csv('../naive_method_v1.0/paper_author_roles.tsv', sep='\t', header=None, names=["paper_index","author_index","role"])

id2authorRole = {}
for i in range(len(author_roles["paper_index"])):
    paperId = author_roles["paper_index"][i]
    authorId = author_roles["author_index"][i]
    if paperId not in id2authorRole:
        id2authorRole[paperId] = {}
    id2authorRole[paperId][authorId] = author_roles["role"][i]
id2authorRole
error_list=[]
i = 0
openai.api_key = "31b8638c9eea48709a596501490f9e88"
openai.api_base = "https://gersteinbiocodeeval-eastus2.openai.azure.com/"
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future
key_bundles = [
    ('aaccba8e27374383beb397ecdc615ee5', "https://biocodeeval-openai.openai.azure.com/"),
    ('31b8638c9eea48709a596501490f9e88', "https://gersteinbiocodeeval-eastus2.openai.azure.com/"),
    ('aaccba8e27374383beb397ecdc615ee5', "https://biocodeeval-openai.openai.azure.com/")
]

    # prompt_construction v1.6
def construct_prompt_cot_agg(reference_dict,paperId,authorTitleList):
    prompt_str = ""
    authors_list = []
    
    ref_example = []

    for index,item in enumerate(reference_dict):
        if index <50:
            ref_example.append("Reference Paper #"+str(index)+"\n"+"Title: "+item["title"])


    for index,item in enumerate(authorTitleList):
        authorId = next(iter(item))
        # authors_list.append("Author #"+str(index)+"\nTitle: "+author_dict[item]["title"])
        authors_list.append("Author #"+str(index)+"\nTitle: "+item[authorId]+"\nRole in research team: "+id2authorRole[paperId][authorId])

    question = papers_sampled[paperId]["abstract"]
    prompt_str = ""
    prompt_str = prompt_str + "Assuming you’re the leading author of a research team and you’re working on a new paper:\n"
    prompt_str = prompt_str + "Now You have gathered some related science papers for reference, each of them contains a title.\n"
    prompt_str = prompt_str + "\n".join(ref_example) + "\n"

    prompt_str = prompt_str + "Authors from your research team have proposed the following titles based on their expertises.\n"
    prompt_str = prompt_str + "\n".join(authors_list) + "\n"

    prompt_str = prompt_str + "Please fully consider the above reference papers and the above proposed title comprehensively and brainstorm one most promising title for a science work, keeping it short and faithful:\n"
    prompt_str = prompt_str + "Title: "
    # prompt_str = prompt_str + "Please consider the above proposed title comprehensively and write a title for the given abstract, keeping it short and faithful:\n"
    # prompt_str = prompt_str + "Title: "

    return prompt_str


id2result_multi_agent = {}
with open("multi_agent_result_method_3.tsv", "wb", buffering=0) as out_file:
# with open("naive_method_v1.0/zero_shot_results.tsv", "wb", buffering=0) as out_file:
    for indice,paperId in enumerate(id2result):
        if paperId in id2citations:
            prompt = construct_prompt_cot_agg(id2citations[paperId],paperId,id2result[paperId])
            # print(prompt)
            # break
            result = run_openai(prompt,key_bundles,i,indice)
            print(indice, result)
            id2result_multi_agent[paperId] = result
            write_str = bytes(str(paperId)+"\t"+result+"\n", 'utf-8')
            out_file.write(write_str)


NameError: name 'id2result' is not defined

In [11]:
results = pd.read_csv('zero_shot_results.tsv', sep='\t', header=None, names=["paper_index","result"])
id2result_multi_agent = {}
for i in range(len(results["result"])):
    id2result_multi_agent[results["paper_index"][i]] = results["result"][i]

from rouge import Rouge
import evaluate
def postprocess_text_generation(preds, labels):
    preds = [pred.replace("\"","").strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def create_metric_rouge():
    rouge_metric = evaluate.load('rouge')
    def compute_metrics(decoded_preds, decoded_labels):
        decoded_preds, decoded_labels = postprocess_text_generation(decoded_preds, decoded_labels)
        result_rouge = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)
        result = {"rouge-1" : result_rouge["rouge1"], "rouge-L" : result_rouge["rougeL"]}
        return result
    return compute_metrics
pred = []
gold = []
for i in id2citations:
    if i in papers_sampled and len(id2citations[i])>0:
        pred.append(id2result_multi_agent[i])
        gold.append(papers_sampled[i]["title"])
# print(pred)
metric = create_metric_rouge()
print(metric(pred, gold))


{'rouge-1': 0.09916574259883223, 'rouge-L': 0.09413915233063627}


In [12]:
error_list=[]
i = 0
openai.api_key = "0de53d4ada864cd18e3771d16551aee1"
openai.api_base = "https://biocodeeval-openai.openai.azure.com/"
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future
key_bundles = [
    ('0de53d4ada864cd18e3771d16551aee1', "https://biocodeeval-openai.openai.azure.com/"),
    ('0de53d4ada864cd18e3771d16551aee1', "https://biocodeeval-openai.openai.azure.com/"),
    ('0de53d4ada864cd18e3771d16551aee1', "https://biocodeeval-openai.openai.azure.com/")
]


def run_openai(prompt,key_bundles,i,indice):
    my_dict = {}
    my_dict["role"] = "user"
    my_dict["content"] = prompt
    l = []
    l.append(my_dict)
    result = ""
    try:
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
        result = response.choices[0].message["content"].replace('\n', ' ')
        # print(indice+"\t"+result+"\n")
        return result
    except openai.error.Timeout:
        i = i+1
        print("Timeout",indice)
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        try:
            response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
            result = response.choices[0].message["content"].replace('\n', ' ')
            # print(indice+"\t"+result+"\n")
            return result
        except openai.error.Timeout:
            i = i+1
            print("Timeout",indice)
            key_bundle = key_bundles[i%3]
            openai.api_key, openai.api_base = key_bundle
            try:
                response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
                result = response.choices[0].message["content"].replace('\n', ' ')
                # print(indice+"\t"+result+"\n")
                return result
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                error_list.append(i)
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            error_list.append(i)
        #print(result)
    except openai.error.InvalidRequestError:
        print("InvalidRequestError",indice)
        error_list.append(i)
        #print(result)
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        error_list.append(i)
    return result

# G-EVAL 
def construct_prompt_g_eval(source,output,dimension):

    Name = ""
    Description = ""
    Instruction = "Evaluation Steps:\n"
    dimensions_single_inputs = ["novelty"]

    if dimension == "coherence":
        Name = "Coherence"
        Description = "Coherence (1-5) - the collective quality of all sentences. We align this dimension with the DUC quality question of structure and coherence whereby \"the title should be well-structured and well-organized. The title should not just be a heap of related information, but should build from sentence to a coherent body of information about a topic.\"\n"
        Instruction = Instruction + "1. Read the science paper abstract carefully and identify the main topic and key points." + "\n"
        Instruction = Instruction + "2. Read the title and compare it to the science paper abstract. Check if the title covers the main topic and key points of the paper abstract, and if it presents them in a clear and logical order." + "\n"
        Instruction = Instruction + "3. Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest based on the Evaluation Criteria." + "\n"
    elif dimension == "consistency":
        Name = "Consistency"
        Description = "Consistency (1-5) - the factual alignment between the title and the science paper abstract. A factually consistent title contains only statements that are entailed by the source document. Annotators were also asked to penalize titles that contained hallucinated facts. \n"
        Instruction = Instruction + "1. Read the science paper abstract carefully and identify the main facts and details it presents." + "\n"
        Instruction = Instruction + "2. Read the title and compare it to the source abstract. Check if the title contains any factual errors that are not supported by the abstract." + "\n"
        Instruction = Instruction + "3. Assign a score for consistency on a scale of 1 to 5, where 1 is the lowest and 5 is the highest based on the Evaluation Criteria." + "\n"
    elif dimension == "fluency":
        Name = "Fluency (1-3)"
        Description = "Fluency (1-3): the quality of the title in terms of grammar, spelling, punctuation, word choice, and sentence structure.\n"
        Description = Description + "- 1: Poor. The title has many errors that make it hard to understand or sound unnatural.\n"
        Description = Description + "- 2: Fair. The title has some errors that affect the clarity or smoothness of the text, but the main points are still comprehensible.\n"
        Description = Description + "- 3: Good. The title has few or no errors and is easy to read and follow.\n"
        Instruction = ""
    elif dimension == "relevance":
        Name = "Relevance"
        Description = "Relevance (1-5) - selection of important content from the source. The title should include only important information from the source document. Annotators were instructed to penalize titles which contained redundancies and excess information.\n"
        Instruction = Instruction + "1. Read the title and the source document carefully." + "\n"
        Instruction = Instruction + "2. Compare the title to the source document and identify the main points of the science paper." + "\n"
        Instruction = Instruction + "3. Assess how well the title covers the main points of the article, and how much irrelevant or redundant information it contains." + "\n"
        Instruction = Instruction + "4. Assign a relevance score from 1 to 5." + "\n"
    
    elif dimension == "informativeness":
        Name = "Informativeness"
        Description = "Informativeness(1-5) - the amount of information, including concepts, key points and findings, from the abstract. Annotators were instructed to penalize titles which contained under-explored concepts or redundancies and excess information.\n"
        Instruction = Instruction + "1. Read the abstract and the source title carefully." + "\n"
        Instruction = Instruction + "2. Check if the abstract effectively communicate the main topic, key points, and significant findings or conclusions. Assess if the information provided is comprehensive, accurate, and complete." + "\n"
        Instruction = Instruction + "3. A good informative abstract (scored 4 or 5) should convey enough amount of conpcets and key points in a progressive manner, while the relatively poorer ones have vague descriptions and under-explored ideas(scored 1 or 2)." + "\n"
        Instruction = Instruction + "4. Assign a score for informativeness on a scale of 1 to 5, to make the reuslts more clearly distributed, please avoid giving score 4" + "\n"
        
    elif dimension == "novelty":
        Name = "Novelty (1-3)"
        Description = "Novelty (1-3): the uniqueness and originality of the abstract in terms of concept, perspective, and creativity. \n"
        Description = Description + "- 1: Poor. The abstract lacks originality and uniqueness, and it closely resembles existing or common titles. \n"
        Description = Description + "- 2: Fair. The abstract shows some degree of uniqueness and creativity, but it still has elements that are common or predictable. \n"
        Description = Description + "- 3: Good. The abstract is highly original and unique, offering a fresh and creative perspective that stands out from common titles.\n"
        Instruction = "Assign a score for novelty on a scale of 1 to 3"

    prompt_str = ""
    prompt_str = prompt_str + "You will be given one abstract written for a science paper.\n"
    prompt_str = prompt_str + "Your task is to rate the output on one metric.\n"
    prompt_str = prompt_str + "Please make sure you read and understand these instructions carefully. Please keep this document open while reviewing, and refer to it as needed.\n"
    prompt_str = prompt_str + "Evaluation Criteria:\n"
    prompt_str = prompt_str + Description
    prompt_str = prompt_str + Instruction
    if dimension not in dimensions_single_inputs:
        prompt_str = prompt_str + "Main topic:\n"
        prompt_str = prompt_str + source + "\n"
    prompt_str = prompt_str + "Abtract:\n"
    prompt_str = prompt_str + output + "\n"
    prompt_str = prompt_str + "Evaluation Form (scores ONLY):\n"
    prompt_str = prompt_str + "- "+Name+":"

    return prompt_str

dimensions = ["informativeness"]
geval_result = {}


i = 0
for j,id in enumerate(id2citations):
    geval_result[id] = {}
    if True:
        if id in papers_sampled and id in id2result_multi_agent:
            for dimension in dimensions:
                # prompt = construct_prompt_g_eval(papers_sampled[id]["title"],id2result_multi_agent[id],dimension)
                # print(prompt)
                
                prompt = construct_prompt_g_eval(papers_sampled[id]["title"],papers_sampled[id]["abstract"],dimension)
                # break
                result = run_openai(prompt,key_bundles,i,i)
                print(result)
                print(str(j),str(dimension),str(result))
                geval_result[id][dimension] = result
                i = i + 1
        # break
        
geval_avg_scores = {"informativeness":0}
count = 0
for paperId in geval_result:
    if "informativeness" in geval_result[paperId]:
        count = count + 1
        for dimension in dimensions:
            if "1. Fluency (1-3): " in geval_result[paperId][dimension]:
                geval_result[paperId][dimension].replace("geval_result[paperId][dimension]","")
            score = int(geval_result[paperId][dimension][0])
            geval_avg_scores[dimension] = geval_avg_scores[dimension] + score

for d in geval_avg_scores:
    print(d+ "\t" + str(geval_avg_scores[d]/count))
count

3
0 informativeness 3
3
1 informativeness 3
5
2 informativeness 5
3
3 informativeness 3
4
4 informativeness 4
4
5 informativeness 4
3
6 informativeness 3
3
7 informativeness 3
4
8 informativeness 4
4
9 informativeness 4
3
10 informativeness 3
3
11 informativeness 3
3
12 informativeness 3
3
13 informativeness 3
3
14 informativeness 3
3
15 informativeness 3
3
16 informativeness 3
3
17 informativeness 3
4
18 informativeness 4
4
19 informativeness 4
3
20 informativeness 3
3
21 informativeness 3
3
22 informativeness 3
3
23 informativeness 3
3
25 informativeness 3
3
26 informativeness 3
3
27 informativeness 3
3
28 informativeness 3
3
30 informativeness 3
3
31 informativeness 3
4
32 informativeness 4
3
33 informativeness 3
3
34 informativeness 3
3
35 informativeness 3
3
36 informativeness 3
4
37 informativeness 4
3
38 informativeness 3
3
39 informativeness 3
3
40 informativeness 3
4
41 informativeness 4
5
42 informativeness 5
4
43 informativeness 4
4
44 informativeness 4
3
45 informativeness 

97

In [10]:
id2result_multi_agent


{}

In [10]:
geval_avg_scores = {"coherence":0,"consistency":0,"fluency":0,"relevance":0}
count = 0
for paperId in geval_result:
    if "coherence" in geval_result[paperId]:
        count = count + 1
        for dimension in dimensions:
            if "1. Fluency (1-3): " in geval_result[paperId][dimension]:
                geval_result[paperId][dimension].replace("geval_result[paperId][dimension]","")
            score = int(geval_result[paperId][dimension][0])
            geval_avg_scores[dimension] = geval_avg_scores[dimension] + score

for d in geval_avg_scores:
    print(d+ "\t" + str(geval_avg_scores[d]/count))

coherence	4.0777777777777775
consistency	4.311111111111111
fluency	2.0444444444444443
relevance	3.888888888888889


In [63]:
author_roles = pd.read_csv('../naive_method_v1.0/paper_author_roles.tsv', sep='\t', header=None, names=["paper_index","author_index","role"])

id2authorRole = {}
for i in range(len(author_roles["paper_index"])):
    paperId = author_roles["paper_index"][i]
    authorId = author_roles["author_index"][i]
    if paperId not in id2authorRole:
        id2authorRole[paperId] = {}
    id2authorRole[paperId][authorId] = author_roles["role"][i]
id2authorRole

{'130862d54894966552cb85d3ee6f739f885d4989': {40350773: 'Author',
  1705375: 'Author and Leading Researcher',
  1690158: 'Author and Leading Researcher'},
 'e28bdc373de80d7ec0e64631a89e64fbdcdae230': {1737649: 'The role of this author in paper writing could be a literature search specialist or a researcher specializing in search strategies and methodologies.',
  145278906: 'The role of this author in paper writing could be a co-author or contributor in the section discussing the importance of qualitative methods in software engineering research and providing guidelines for their use.',
  6648358: 'The role of this author in paper writing could be focused on the strategy and processes of software product customization in large-scale industrial settings. They could contribute to the discussion and analysis of strategies and processes used in the development of customized software products in industrial settings, particularly in the context of telecom companies in Germany and Sweden. They